<a href="https://colab.research.google.com/github/annamaartensson/dd2424project/blob/issue%2F2b/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [ ]:
path_to_file = tf.keras.utils.get_file('prideandprejudice.txt', 'https://www.gutenberg.org/cache/epub/42671/pg42671.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

725256/725256 [==============================] - 0s 0us/step


In [ ]:
vocabulary = sorted(set(text))
ids_to_chars =
chars_to_ids =

In [87]:
class RNN:

  class Grads:

    def __init__(self, m, K):
      self.b = tf.Variable(tf.zeros_initializer()(shape = (K)))
      self.c = tf.Variable(tf.zeros_initializer()(shape = (K)))
      self.U = tf.Variable(tf.zeros_initializer()(shape = (m, K)))
      self.W = tf.Variable(tf.zeros_initializer()(shape = (m, m)))
      self.V = tf.Variable(tf.zeros_initializer()(shape = (K, m)))

    def compute(self):
      pass

  def __init__(self, seq_length, m, K, sig):
    self.seq_length = seq_length
    self.m = m
    self.K = K
    self.b = tf.Variable(tf.zeros_initializer()(shape = (m)))
    self.c = tf.Variable(tf.zeros_initializer()(shape = (K)))
    self.U = tf.Variable(tf.random_normal_initializer(mean=0.0, stddev=sig)(shape = (m, K)))
    self.W = tf.Variable(tf.random_normal_initializer(mean=0.0, stddev=sig)(shape = (m, m)))
    self.V = tf.Variable(tf.random_normal_initializer(mean=0.0, stddev=sig)(shape = (K, m)))
    self.h0 = tf.Variable(tf.zeros_initializer()(shape = (m)))

  def forwardPass(self, X):
    P = tf.Variable(tf.zeros_initializer()(shape = (self.K, self.seq_length)))
    H = tf.Variable(tf.zeros_initializer()(shape = (self.m, self.seq_length+1)))
    A = tf.Variable(tf.zeros_initializer()(shape = (self.m, self.seq_length)))
    H[:,0].assign(self.h0)
    for t in range(self.seq_length):
      A[:,t].assign(tf.tensordot(self.W, H[:,t], 1) + tf.tensordot(self.U, X[:,t], 1) + self.b)
      H[:,t+1].assign(tf.math.tanh(A[:,t]))
      P[:,t].assign(tf.raw_ops.Softmax(logits = (tf.tensordot(self.V, H[:,t+1], 1) + self.c)))
    return P, H, A

  def backwardPass(self, X, Y, P, H, A):
    debug=False
    grads = RNN.Grads(self.m, self.K)

    G = tf.transpose(Y-P) * -1
    if debug:
      print(G)

    #dL/dc
    grads.c.assign(tf.reduce_sum(G, axis=0))
    if debug:
      print(grads.c)

    #dL/dV
    for i in range(self.seq_length):
      grads.V.assign(grads.V+tf.tensordot(G[i,:],H[:,i],0))
    if debug:
      print(grads.V)

    #dL/dh and dL/da, in grad_H and grad_A each row represents a timestep
    grad_H = tf.Variable(tf.zeros_initializer()(shape = (self.seq_length, self.m)))
    grad_A = tf.Variable(tf.zeros_initializer()(shape = (self.seq_length, self.m)))
    grad_H[-1,:].assign(tf.tensordot(tf.reshape(G[-1,:], (1,-1)),self.V,1))
    for i in range(self.seq_length):
      grad_A[-i,:].assign(tf.tensordot(tf.reshape(grad_H[-i,:],(1,-1)),tf.linalg.diag(tf.ones(self.m)-tf.math.square(tf.math.tanh(A[:,-i]))),1))
      grad_H[-i-1,:].assign(tf.tensordot(tf.reshape(G[-i-1,:], (1,-1)),self.V,1)+tf.tensordot(tf.reshape(grad_A[-i,:], (1,-1)),self.W,1))
    grad_A[0,:].assign(tf.tensordot(tf.reshape(grad_H[0,:],(1,-1)),tf.linalg.diag(tf.ones(self.m)-tf.math.square(tf.math.tanh(A[:,0]))),1))

    #dL/db
    grads.b.assign(tf.reduce_sum(grad_A, axis=0))

    #dL/dW
    for i in range(self.seq_length):
      grads.W.assign(grads.W+tf.tensordot(grad_A[i,:],H[:,i],0))

    #dL/dU
    for i in range(self.seq_length):
      grads.U.assign(grads.W+tf.tensordot(grad_A[i,:],X[:,i],0))

    return grads

  def loss(self, X, Y, P):
    pass

  def train(self, eta, steps):
    #TODO: replace with real data/batches
    X = tf.Variable(tf.zeros_initializer()(shape = (80, 25)))
    Y = tf.Variable(tf.zeros_initializer()(shape = (80, 25)))
    P, H, A = self.forwardPass(X)
    grads = self.backwardPass(X, Y, P, H, A)


In [88]:
rnn = RNN(25, 100, 80, 0.01)
x = tf.Variable(tf.zeros_initializer()(shape = (80, 25)))
rnn.train(1e-2,10)

ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (80,), and the assigned value shape (100,) are incompatible.

In [31]:
import numpy as np
Y = tf.Variable(np.array([[1,2],[3,4]]))
P = tf.Variable(np.array([[1,1],[1,1]]))


tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int64)


<tf.Variable 'UnreadVariable' shape=(2, 2) dtype=int64, numpy=
array([[2, 3],
       [4, 5]])>

In [73]:
a=[1,2,3]
tf.linalg.diag(a)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]], dtype=int32)>